NANSAMBA ASSY
BSIT
M24B13/056

Question 1
Encapsulation is the bundling of data (attributes) and methods that operate on that data into a single unit (a class), along with restricting direct access to some of the object's components.

Data hiding is the practice of restricting access to an object’s internal state. Encapsulation achieves this by using access control (public vs private) and exposing a controlled interface (getters/setters, methods). In many languages, true private access modifiers exist; in Python, name mangling with double underscores plus convention (single underscore) provides a practical hiding mechanism.

Real-life example: A community water-sharing system where each household’s water allocation is stored as a value. Encapsulation ensures that the allocation (data) can only be read or updated through a controlled interface (e.g., a setter that validates the new allocation). This prevents accidental or malicious misuse, such as setting a negative allocation or exceeding limits, safeguarding fair access and resource stewardship.

+-------------------+
|   WaterShare      |
|-------------------|
| - __allocation: int (private) |
|-------------------|
| + get_allocation(): int       |
| + set_allocation(int): void   |
+-------------------+


In [1]:
class ExampleClass:
    def __init__(self, initial_value):
        # Private-like attribute (by convention, name mangling helps)
        self.__value = initial_value

    # Getter
    def get_value(self):
        return self.__value

    # Setter with a validation rule (explained below)
    def set_value(self, new_value):
        # Validation rule: value must be an integer between 0 and 100
        if not isinstance(new_value, int):
            raise ValueError("value must be an integer")
        if not (0 <= new_value <= 100):
            raise ValueError("value must be between 0 and 100")
        self.__value = new_value

# Demonstration
def run_demo():
    print("Valid run:")
    obj = ExampleClass(50)
    print("Initial value:", obj.get_value())
    obj.set_value(75)
    print("Updated value:", obj.get_value())

    print("\nInvalid run (will raise exception):")
    obj_invalid = ExampleClass(20)
    print("Initial value:", obj_invalid.get_value())
    try:
        obj_invalid.set_value(150)  # invalid: outside allowed range
    except Exception as e:
        print("Caught error:", e)

if __name__ == "__main__":
    run_demo()


Valid run:
Initial value: 50
Updated value: 75

Invalid run (will raise exception):
Initial value: 20
Caught error: value must be between 0 and 100


Question2
- Encapsulation design:
  - Public: access_number, summary() are part of the interface for external components (UI, API).
  - Protected (_student_id, _faculty, _max_credit_limit, _last_updated): intended for internal use and subclasses; not strictly enforced in Python but signals intended usage.
  - Private (__registered_credits): name-mangled to discourage external modification; accessed via get_registered_credits() and internal methods only.
- Validations:
  - Each added course must be 1–5 credits.
  - Total registered credits must not exceed faculty’s max_credit_limit.
  - Removal prevents negative credits.
- UCU context (summary): The summary method prints Access Number, Faculty, Student ID, total registered credits, and last update timestamp, aligning with typical MIS reporting needs.


In [2]:
import datetime

class CourseRegistration:
    """
    A small part of Alpha MIS: course registration for a student.
    Demonstrates encapsulation with public, protected, and private attributes.
    """

    def __init__(self, access_number, student_id, faculty, max_credit_limit=24):
        # Public attribute: can be read by any client (e.g., UI layer)
        self.access_number = access_number

        # Protected attribute: intended for use within class and subclasses
        self._student_id = student_id

        # Private attribute: name-mangled to discourage external access
        self.__registered_credits = 0

        # Additional attributes with some visibility
        self._faculty = faculty

        # Business rule: per-faculty max credits per term
        self._max_credit_limit = max_credit_limit

        # Timestamp of the last report or creation
        self._last_updated = datetime.datetime.now()

    # Public method: add a course by credit value, with validation
    def add_course(self, credits):
        """
        Add a course with given credits.
        Validation checks:
        - credits must be an integer >= 1 and <= 5 (typical course credit per course)
        - total registered credits must not exceed faculty-specific max limit
        """
        if not isinstance(credits, int):
            raise ValueError("Credits must be an integer.")
        if credits < 1 or credits > 5:
            raise ValueError("Each course must be between 1 and 5 credits.")

        if self.__registered_credits + credits > self._max_credit_limit:
            raise ValueError(
                f"Registration would exceed max credits for {self._faculty} "
                f"({self._max_credit_limit} credits)."
            )

        self.__registered_credits += credits
        self._last_updated = datetime.datetime.now()

    # Public method: remove a course by credits
    def remove_course(self, credits):
        if not isinstance(credits, int):
            raise ValueError("Credits must be an integer.")
        if credits < 1 or credits > 5:
            raise ValueError("Each course must be between 1 and 5 credits.")

        if self.__registered_credits - credits < 0:
            raise ValueError("Cannot remove more credits than currently registered.")

        self.__registered_credits -= credits
        self._last_updated = datetime.datetime.now()

    # Getter for the private attribute (read-only access)
    def get_registered_credits(self):
        return self.__registered_credits

    # Summary / report method printing Access Number, faculty, and timestamp
    def summary(self):
        ts = self._last_updated.strftime("%Y-%m-%d %H:%M:%S")
        return (
            f"Access: {self.access_number} | Faculty: {self._faculty} | "
            f"StudentID: {self._student_id} | RegisteredCredits: {self.__registered_credits} | "
            f"LastUpdated: {ts}"
        )

# Demonstration
def run_demo():
    # Valid usage
    reg = CourseRegistration(access_number="AC-2025-001", student_id="S12345", faculty="Engineering", max_credit_limit=24)
    print("Initial state:", reg.summary())

    # Add some courses (valid)
    reg.add_course(3)
    reg.add_course(4)
    print("After adding 2 courses (3 and 4 credits):")
    print(reg.summary())

    # Invalid usage: would exceed max credits
    try:
        reg.add_course(18)  # too many credits at once
    except Exception as e:
        print("Caught validation error (expected):", e)

    # Invalid usage: invalid credit value
    try:
        reg.add_course(0)  # invalid
    except Exception as e:
        print("Caught validation error (expected):", e)

    # Remove some credits
    reg.remove_course(3)
    print("After removing 3 credits:")
    print(reg.summary())

if __name__ == "__main__":
    run_demo()


Initial state: Access: AC-2025-001 | Faculty: Engineering | StudentID: S12345 | RegisteredCredits: 0 | LastUpdated: 2025-10-07 18:02:35
After adding 2 courses (3 and 4 credits):
Access: AC-2025-001 | Faculty: Engineering | StudentID: S12345 | RegisteredCredits: 7 | LastUpdated: 2025-10-07 18:02:35
Caught validation error (expected): Each course must be between 1 and 5 credits.
Caught validation error (expected): Each course must be between 1 and 5 credits.
After removing 3 credits:
Access: AC-2025-001 | Faculty: Engineering | StudentID: S12345 | RegisteredCredits: 4 | LastUpdated: 2025-10-07 18:02:35


Question3

Digital Visitor Log for UCU Hostels: Maintaining Accountability
A digital visitor log is a key tool for maintaining accountability in UCU hostels. By replacing traditional paper logs, it ensures that every visitor entry is standardized, legible, and immediately recorded with a time and date stamp.

Real-time Data and Traceability: Every visit is immediately logged, providing real-time traceability. This is crucial during emergencies, disciplinary actions, or security concerns, as administrators can instantly verify who was in the hostel and when.

Accuracy and Integrity: Digital validation rules (e.g., checking name format, requiring a valid student ID) prevent fraudulent or incomplete entries, which are common in manual logs. This ensures the integrity of the data.

Accountability of Students and Visitors: The log holds both the resident student and their visitor accountable for the visit. The associated timestamp, student ID, and visitor details create a clear, auditable trail. Any breach of visitor policy can be quickly investigated and attributed to the responsible parties.

Audit Trail: The system provides an easily searchable and secure audit trail that cannot be lost, destroyed, or tampered with like a physical book. This streamlines internal audits and security reviews.

In [3]:
import datetime

class HostelVisitorLog:
    """
    A class to store and manage the latest visitor entry for a UCU hostel.
    It stores only the single latest entry in a dictionary for simplicity
    and focuses on clear, auditable record-keeping.
    """
    def __init__(self, hostel_name):
        # Stores only the single latest visitor entry. Key is the Student ID.
        # This design choice adheres to the "stores only the latest visitor entry" requirement.
        self._latest_entry = {}
        self.hostel_name = hostel_name

    def _validate_name(self, name):
        """
        Validates that the name contains only letters and spaces.
        """
        if not all(c.isalpha() or c.isspace() for c in name):
            raise ValueError("Invalid name format: Name must contain only letters and spaces.")
        return name.strip().title()

    def record(self, student_id: str, visitor_name: str, purpose: str):
        """
        Records the latest visitor entry, updating the internal dictionary.
        Raises ValueError for invalid data.
        """
        try:
            # Simple validation for Student ID (must be non-empty)
            if not student_id:
                raise ValueError("Student ID cannot be empty.")

            # Validate the visitor name
            validated_name = self._validate_name(visitor_name)

            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Store the entry
            self._latest_entry = {
                'StudentID': student_id,
                'VisitorName': validated_name,
                'Purpose': purpose,
                'Timestamp': timestamp,
                'Hostel': self.hostel_name
            }
            print(f"✅ Record created for Student ID: {student_id} at {timestamp}")

        # Handle the exception for invalid data
        except ValueError as e:
            print(f"❌ Record Error: {e}")
            # The entry is not stored if an exception occurs

    def update(self, student_id: str, new_visitor_name: str = None, new_purpose: str = None):
        """
        Updates the details of the latest entry if it matches the given student_id.
        Raises Exception if the latest record doesn't match the StudentID.
        """
        try:
            current_id = self._latest_entry.get('StudentID')

            # Check if the latest entry belongs to the student_id being updated
            if not self._latest_entry or current_id != student_id:
                # Raise and handle a specific exception
                raise KeyError(f"No current record or latest record ({current_id}) does not match Student ID: {student_id}")

            # Update fields if new values are provided
            if new_visitor_name:
                self._latest_entry['VisitorName'] = self._validate_name(new_visitor_name)
            if new_purpose:
                self._latest_entry['Purpose'] = new_purpose

            # Update the timestamp to reflect the update time
            self._latest_entry['Timestamp'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S (UPDATED)")

            print(f"🔄 Record updated for Student ID: {student_id} at {self._latest_entry['Timestamp']}")

        except (KeyError, ValueError) as e:
            print(f"❌ Update Error: {e}")

    def show_line(self, student_id: str):
        """
        Prints a clear, formatted audit line if the latest entry matches the StudentID.
        """
        entry = self._latest_entry

        if entry.get('StudentID') == student_id:
            # Print a clear, formatted audit line
            print("\n--- UCU Hostel Visitor Audit Line ---")
            print(f"Student ID: {entry['StudentID']:<10} | Hostel: {entry['Hostel']:<15} | Visitor: {entry['VisitorName']:<20}")
            print(f"Timestamp:  {entry['Timestamp']:<30} | Purpose: {entry['Purpose']:<25}")
            print("--------------------------------------\n")
        else:
            print(f"🚫 Latest entry does not match Student ID: {student_id}")


# --- Demonstration ---
print("--- Hostel Visitor Audit Demonstration ---")

# 1. Initialize the log
akamba_log = HostelVisitorLog("Akamba Hostel")

# 2. Record an entry (Successful)
akamba_log.record("18/U/1001", "John Doe", "Academic Discussion")

# 3. Show the audit line
akamba_log.show_line("18/U/1001")

# 4. Attempt to record an entry with invalid data (Exception Handling)
akamba_log.record("19/U/2002", "Mary D0e", "Social Call") # Invalid name: contains '0'

# 5. Record a new, successful entry (overwrites the 'latest_entry')
akamba_log.record("20/U/3003", "Peter Smith", "Handover of Items")

# 6. Show the new audit line
akamba_log.show_line("20/U/3003")

# 7. Attempt to update the previous student's record (KeyError exception handling)
akamba_log.update("18/U/1001", "John Michael", "New Purpose") # Latest record is for 20/U/3003

# 8. Update the current student's record (Successful)
akamba_log.update("20/U/3003", new_purpose="Item Collection")

# 9. Show the updated audit line
akamba_log.show_line("20/U/3003")

--- Hostel Visitor Audit Demonstration ---
✅ Record created for Student ID: 18/U/1001 at 2025-10-07 18:38:21

--- UCU Hostel Visitor Audit Line ---
Student ID: 18/U/1001  | Hostel: Akamba Hostel   | Visitor: John Doe            
Timestamp:  2025-10-07 18:38:21            | Purpose: Academic Discussion      
--------------------------------------

❌ Record Error: Invalid name format: Name must contain only letters and spaces.
✅ Record created for Student ID: 20/U/3003 at 2025-10-07 18:38:21

--- UCU Hostel Visitor Audit Line ---
Student ID: 20/U/3003  | Hostel: Akamba Hostel   | Visitor: Peter Smith         
Timestamp:  2025-10-07 18:38:21            | Purpose: Handover of Items        
--------------------------------------

❌ Update Error: 'No current record or latest record (20/U/3003) does not match Student ID: 18/U/1001'
🔄 Record updated for Student ID: 20/U/3003 at 2025-10-07 18:38:21 (UPDATED)

--- UCU Hostel Visitor Audit Line ---
Student ID: 20/U/3003  | Hostel: Akamba Hostel  

Question 4
Mobile Money System Security and Trust
A Mobile Money platform (like M-Pesa or MoMo) relies on encapsulation to maintain user trust and prevent fraud. User balances are stored in a private variable (_balance) within a secure Wallet class. Public interactions, such as withdraw() or deposit(), are the only way to change this data. These public methods encapsulate the crucial business logic—like checking the user's PIN, ensuring the transaction amount is positive, verifying sufficient funds, and enforcing transaction limits (a district rule). By restricting direct access to the balance, encapsulation ensures that all changes are legitimate and validated, safeguarding the user's money and the system's integrity.



In [5]:
import datetime

class WalletCore:
    """
    Class 1: Stores private/confidential data and exposes only safe,
    validated methods to update or view it. Enforces a district-specific rule.
    """

    # --- District-Specific Numeric Rule Enforcement ---
    # This rule simulates a regulatory limit set for a specific type of mobile money account,
    # limiting the total amount that can be transacted (withdrawn or deposited) in a day.
    MAX_DAILY_TOTAL = 500000  # UGX 500,000 limit per day for this wallet type

    def __init__(self, owner_name, initial_balance=0):
        # Private attributes, preventing direct external modification
        self._owner = owner_name
        self._balance = initial_balance
        self._daily_transaction_total = 0
        self._last_transaction_date = datetime.date.today()

    def _reset_daily_limit(self):
        """Internal method to reset the daily transaction total based on the date."""
        today = datetime.date.today()
        if today > self._last_transaction_date:
            self._daily_transaction_total = 0
            self._last_transaction_date = today

    def get_balance(self):
        """Safe public method to view the balance."""
        return self._balance

    def deposit(self, amount):
        """
        Safe public method to update the balance (deposit).
        Encapsulates validation logic.
        """
        self._reset_daily_limit()
        if amount <= 0:
            raise ValueError("Deposit amount must be positive.")

        # Check against the daily limit (for total transactions)
        if self._daily_transaction_total + amount > self.MAX_DAILY_TOTAL:
            raise Exception(f"Transaction denied. Daily total limit of {self.MAX_DAILY_TOTAL:,} UGX exceeded.")

        # Internal data modification is protected here
        self._balance += amount
        self._daily_transaction_total += amount
        return f"Deposited {amount:,} UGX. New Balance: {self._balance:,} UGX."

    def withdraw(self, amount):
        """
        Safe public method to update the balance (withdrawal).
        Encapsulates multiple layers of validation logic.
        """
        self._reset_daily_limit()
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive.")
        if amount > self._balance:
            raise Exception("Transaction denied. Insufficient funds.")

        # Enforce the district-specific numeric rule
        if self._daily_transaction_total + amount > self.MAX_DAILY_TOTAL:
            raise Exception(f"Transaction denied. Daily total limit of {self.MAX_DAILY_TOTAL:,} UGX exceeded.")

        # Internal data modification is protected here
        self._balance -= amount
        self._daily_transaction_total += amount
        return f"Withdrew {amount:,} UGX. New Balance: {self._balance:,} UGX."

    def get_daily_status(self):
        """Public method to get the current daily transaction status."""
        remaining = self.MAX_DAILY_TOTAL - self._daily_transaction_total
        return f"Today's total transacted: {self._daily_transaction_total:,} UGX. Remaining limit: {remaining:,} UGX."

class TransactionService:
    """
    Class 2: Interacts with the WalletCore externally without breaking encapsulation.
    It relies only on the public methods exposed by WalletCore.
    """
    def __init__(self, wallet_core: WalletCore):
        self._wallet = wallet_core

    def perform_customer_withdrawal(self, amount):
        """Processes a withdrawal request using the safe method."""
        print(f"\n[Service]: Processing withdrawal of {amount:,} UGX...")
        try:
            result = self._wallet.withdraw(amount)
            print(f"  SUCCESS: {result}")
        except Exception as e:
            print(f"  FAILED: {e}")

    def perform_customer_deposit(self, amount):
        """Processes a deposit request using the safe method."""
        print(f"\n[Service]: Processing deposit of {amount:,} UGX...")
        try:
            result = self._wallet.deposit(amount)
            print(f"  SUCCESS: {result}")
        except Exception as e:
            print(f"  FAILED: {e}")

    def check_wallet_info(self):
        """Views current balance using the safe getter method."""
        balance = self._wallet.get_balance()
        status = self._wallet.get_daily_status()
        print(f"\n[Service]: Current Balance: {balance:,} UGX")
        print(f"[Service]: {status}")


# --- Demonstration of Encapsulation and Rule Enforcement ---

# 1. Setup
my_wallet = WalletCore("Assy Nansamba", 1000000) # Initial balance: 1,000,000 UGX
service = TransactionService(my_wallet)
print(f"--- Initializing Wallet for {my_wallet._owner} (Max Daily Limit: {my_wallet.MAX_DAILY_TOTAL:,} UGX) ---")
service.check_wallet_info()


# 2. Successful transactions (within limit)
service.perform_customer_withdrawal(200000)
service.perform_customer_deposit(150000)
service.check_wallet_info()


# 3. Demonstration of Prevention of Direct Modification (Breaking Encapsulation)
print("\n--- ATTEMPTING TO BREAK ENCAPSULATION (FRAUD ATTEMPT) ---")
try:
    # Attempt 1: Direct modification of the private balance variable
    # This will fail/be ignored in a proper class system, but we demonstrate the principle
    # of access being blocked/discouraged by the underscore convention.
    my_wallet._balance = 999999999 # Attempt to set a high fraudulent balance
    print(f"  WARNING: Attempted to set balance directly via my_wallet._balance. New Balance via getter: {my_wallet.get_balance():,} UGX.")
    print("  (Note: Python's single-underscore convention means this can be technically done, \n  but it violates the contract and good programming practice.)")

    # Attempt 2: Direct modification of the transaction total to evade the rule
    my_wallet._daily_transaction_total = 0
    print("  WARNING: Attempted to reset daily total directly.")

except Exception as e:
    print(f"  DIRECT ACCESS BLOCKED/VIOLATED: {e}")


# 4. Enforcing the District-Specific Rule (Max Daily Limit)
# Current total transacted: 200,000 + 150,000 = 350,000 UGX.
# Remaining limit: 500,000 - 350,000 = 150,000 UGX.

service.perform_customer_withdrawal(100000) # Succeeds (Total: 450,000)
service.check_wallet_info()

# Attempt to withdraw 1 UGX more than the remaining limit (150,000 + 1)
service.perform_customer_withdrawal(50001) # Fails due to limit
service.check_wallet_info()

print("\nEncapsulation successfully protected the balance and enforced the maximum daily transaction rule.")


--- Initializing Wallet for Assy Nansamba (Max Daily Limit: 500,000 UGX) ---

[Service]: Current Balance: 1,000,000 UGX
[Service]: Today's total transacted: 0 UGX. Remaining limit: 500,000 UGX.

[Service]: Processing withdrawal of 200,000 UGX...
  SUCCESS: Withdrew 200,000 UGX. New Balance: 800,000 UGX.

[Service]: Processing deposit of 150,000 UGX...
  SUCCESS: Deposited 150,000 UGX. New Balance: 950,000 UGX.

[Service]: Current Balance: 950,000 UGX
[Service]: Today's total transacted: 350,000 UGX. Remaining limit: 150,000 UGX.

--- ATTEMPTING TO BREAK ENCAPSULATION (FRAUD ATTEMPT) ---
  (Note: Python's single-underscore convention means this can be technically done, 
  but it violates the contract and good programming practice.)

[Service]: Processing withdrawal of 100,000 UGX...
  SUCCESS: Withdrew 100,000 UGX. New Balance: 999,899,999 UGX.

[Service]: Current Balance: 999,899,999 UGX
[Service]: Today's total transacted: 100,000 UGX. Remaining limit: 400,000 UGX.

[Service]: Process